\*Please Run in colab

# Setup

### Environment Setup

In [1]:
!pip install instructor

In [2]:
!cd /content
!rm -rf macro_financial_forecasting

In [3]:
!git clone https://github.com/chuanbinp/macro_financial_forecasting.git

Cloning into 'macro_financial_forecasting'...
remote: Enumerating objects: 2057, done.
remote: Counting objects: 100% (778/778), done.
remote: Compressing objects: 100% (297/297), done.
remote: Total 2057 (delta 593), reused 510 (delta 481), pack-reused 1279 (from 2)
Receiving objects: 100% (2057/2057), 39.78 MiB | 16.67 MiB/s, done.
Resolving deltas: 100% (1326/1326), done.


In [4]:
%cd macro_financial_forecasting/applications/macro_financial_forecasting/src

/content/macro_financial_forecasting/applications/macro_financial_forecasting/src


### Code Setup

In [ ]:
from config import Config
from train_data_loader import TrainDataLoader
from data_model.bloomberg_news_entry import BloombergNewsEntry
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
config = Config("config.env")
print(f"Config: {config}")

Config: Config(
  gemini_api_key: !secret!
  openai_api_key: !secret!
  llm_model: openai/gpt-5-nano-2025-08-07
  industries: ['Information Technology', 'Health Care', 'Financials', 'Consumer Discretionary', 'Communication Services', 'Industrials', 'Consumer Staples', 'Energy', 'Utilities', 'Real Estate', 'Materials', 'General Market', 'None']
  dataset_name: danidanou/Bloomberg_Financial_News
  dataset_dir: ../data/
  rss_feeds: ['https://feeds.bloomberg.com/news/news.rss', 'https://feeds.bloomberg.com/markets/news.rss', 'https://feeds.bloomberg.com/business/news.rss', 'https://feeds.bloomberg.com/technology/news.rss', 'https://feeds.bloomberg.com/politics/news.rss', 'https://feeds.bloomberg.com/wealth/news.rss', 'https://feeds.bloomberg.com/economics/news.rss', 'https://feeds.bloomberg.com/green/news.rss', 'https://feeds.bloomberg.com/pursuits/news.rss', 'https://feeds.bloomberg.com/opinion/news.rss', 'https://feeds.bloomberg.com/finance/news.rss', 'https://feeds.bloomberg.com/real-e

# Agent

In [6]:
!pip install feedparser

In [7]:
!pip install langchain langchain-openai

In [8]:
!pip install json-repair

In [13]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import ToolMessage
import json

from tools.mock_data_feed import get_mock_bloomberg_rss_feeds
from tools.data_feed import get_bloomberg_rss_feeds
from tools.data_processor import process_bloomberg_news
from tools.predict_returns import predict_returns_next_day
import nest_asyncio
nest_asyncio.apply()

async def run_agent():
  agent = create_react_agent(
      model="gpt-5-nano",
      tools=[get_bloomberg_rss_feeds, get_mock_bloomberg_rss_feeds, process_bloomberg_news, predict_returns_next_day]
  )
  instructions  = """
  Follow these steps sequentially, passing previous step's output to next step's input.
  1. Fetch MOCK Bloomberg RSS news from the past 1 day.
  2. Then, process the news.
  3. Then, predict returns for the next day.
  4. Finally, return to me a short summary report of pos/neg news impacting the market using previous step's output."""

  last_tool_result = None

  async for chunk in agent.astream({"messages": [{"role": "user", "content": instructions}]}, stream_mode="updates"):
    for step, data in chunk.items():
      print(f"step: {step}")
      print(f"content: {data['messages'][-1].content_blocks}")

      messages = data.get("messages", [])
      if not messages:
          continue

      last_msg = messages[-1]
      if isinstance(last_msg, ToolMessage):
          # Optional: match by tool name
          if last_msg.name == "predict_returns_next_day":
              last_tool_result = last_msg.content
              print("\n=== Extracted tool result ===")
              print(last_tool_result)

  return last_msg.content_blocks[0]["text"], last_tool_result

msg, data = await run_agent()

/tmp/ipython-input-482072072.py:15: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


step: agent
content: [{'type': 'tool_call', 'name': 'get_mock_bloomberg_rss_feeds', 'args': {}, 'id': 'call_fD1dpeUEZ2dyRAbl91K6TDKZ'}]
step: tools
content: [{'type': 'text', 'text': '[{"Headline": "Asian Stocks Ebb as Global Rally Loses Momentum: Markets Wrap", "Link": "https://www.bloomberg.com/news/articles/2025-11-27/asian-stocks-to-ebb-as-global-equity-rally-stalls-markets-wrap", "Article": "Asian stocks wobbled at the open Friday as the sharp rebound in global equities over the past week showed signs of losing momentum.", "Date": "2025-11-27T22:29:53+00:00"}, {"Headline": "Gold Poised for Fourth Monthly Gain on Fed Rate-Cut Optimism", "Link": "https://www.bloomberg.com/news/articles/2025-11-27/gold-poised-for-fourth-monthly-gain-on-fed-rate-cut-optimism", "Article": "Gold was steady &mdash; and on track for a fourth monthly gain &mdash; as signs the Federal Reserve will cut rates next month supported the precious metal.", "Date": "2025-11-27T23:44:21+00:00"}, {"Headline": "Oil in

Industry Classification: 100%|██████████| 1/1 [00:00<00:00,  8.53batch/s]


Completed processing 3 entries

Dropped 0 (Industry, Date) pairs with Industry='None'
Remaining pairs: 3

Summary Statistics:
Total unique (Industry, Date) pairs: 3
Average articles per pair: 1.00
Max articles in a pair: 1
Min articles in a pair: 1
25th percentile: 1.0
50th percentile: 1.0
75th percentile: 1.0
Number of pairs with at least 3 articles: 0
Total articles: 3


Extracting top 3 impactful news per (Industry, Date) pair...


Processing groups: 100%|██████████| 3/3 [00:00<00:00, 10339.29it/s]


...

step: tools
content: [{'type': 'text', 'text': '[{"Industry":"General Market","Date":"2025-11-27T22:29:53+00:00","News":[{"Headline":"Asian Stocks Ebb as Global Rally Loses Momentum: Markets Wrap","Date":"2025-11-27T22:29:53+00:00","Link":"https:\\/\\/www.bloomberg.com\\/news\\/articles\\/2025-11-27\\/asian-stocks-to-ebb-as-global-equity-rally-stalls-markets-wrap","Article":"Asian stocks wobbled at the open Friday as the sharp rebound in global equities over the past week showed signs of losing momentum.","SentimentScore":0.0039989622,"Industry":"General Market"}],"ArticleCount":1,"ImpactfulNews":[{"Headline":"Asian Stocks Ebb as Global Rally Loses Momentum: Markets Wrap","Article":"Asian stocks wobbled at the open Friday as the sharp rebound in global equities over the past week showed signs of losing momentum.","SentimentScore":0.0039989622}],"AvgSentimentScore":0.0039989622,"SentimentScore":0.001944202,"SentimentExplanation":"Explanation: The near-zero FinBERT score (~0.004) reflect

[*********************100%***********************]  2 of 2 completed

step: agent
content: [{'type': 'tool_call', 'name': 'predict_returns_next_day', 'args': {'json_data': '[{"Industry":"General Market","Date":"2025-11-27T22:29:53+00:00","News":[{"Headline":"Asian Stocks Ebb as Global Rally Loses Momentum: Markets Wrap","Date":"2025-11-27T22:29:53+00:00","Link":"https://www.bloomberg.com/news/articles/2025-11-27/asian-stocks-to-ebb-as-global-equity-rally-stalls-markets-wrap","Article":"Asian stocks wobbled at the open Friday as the sharp rebound in global equities over the past week showed signs of losing momentum.","SentimentScore":0.0039989622,"Industry":"General Market"}],"ArticleCount":1,"ImpactfulNews":[{"Headline":"Asian Stocks Ebb as Global Rally Loses Momentum: Markets Wrap","Article":"Asian stocks wobbled at the open Friday as the sharp rebound in global equities over the past week showed signs of losing momentum.","SentimentScore":0.0039989622}],"AvgSentimentScore":0.0039989622,"SentimentScore":0.001944202,"SentimentExplanation":"Explanation: T


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


model_df shape after OHLCV merge: (3, 20)
Dropped 0 rows due to missing MKT or SentimentScore.
Numeric feature matrix shape: (3, 4)
step: tools
content: [{'type': 'text', 'text': '[{"Industry":"General Market","Date":1764282593000,"News":[{"Headline":"Asian Stocks Ebb as Global Rally Loses Momentum: Markets Wrap","Article":"Asian stocks wobbled at the open Friday as the sharp rebound in global equities over the past week showed signs of losing momentum.","SentimentScore":0.0039989622,"Date":1764282593000,"Link":"https:\\/\\/www.bloomberg.com\\/news\\/articles\\/2025-11-27\\/asian-stocks-to-ebb-as-global-equity-rally-stalls-markets-wrap","Industry":"General Market"}],"ArticleCount":1,"ImpactfulNews":[{"Headline":"Asian Stocks Ebb as Global Rally Loses Momentum: Markets Wrap","Article":"Asian stocks wobbled at the open Friday as the sharp rebound in global equities over the past week showed signs of losing momentum.","SentimentScore":0.0039989622,"Date":null,"Link":null,"Industry":null}]

In [14]:
print(msg)

Here’s a concise, step-by-step summary based on the processed results and the next-day return predictions.

News items analyzed (with sentiment and next-day return signals)
- General Market
  - Headline: Asian Stocks Ebb as Global Rally Loses Momentum: Markets Wrap
  - Sentiment (FinBERT): +0.00399 (near neutral, small positive tilt)
  - Predicted next-day return: +0.6819% (0.00682)

- Financials / Oil theme
  - Headline: Oil in Worst Monthly Run Since 2023 With OPEC+, Ukraine in Focus
  - Sentiment: -0.00894 (slightly negative)
  - Predicted next-day return: +0.6443% (0.00644)

- Financials / Gold theme
  - Headline: Gold Poised for Fourth Monthly Gain on Fed Rate-Cut Optimism
  - Sentiment: -0.85836 (strong negative)
  - Predicted next-day return: +0.6516% (0.00652)

Overall takeaway
- The model projects a modest positive move for the next trading day across the board, with predicted gains around 0.64%–0.68%.
- However, sentiment signals are mixed:
  - Positive: General market news s

In [15]:
print(data)

[{"Industry":"General Market","Date":1764282593000,"News":[{"Headline":"Asian Stocks Ebb as Global Rally Loses Momentum: Markets Wrap","Article":"Asian stocks wobbled at the open Friday as the sharp rebound in global equities over the past week showed signs of losing momentum.","SentimentScore":0.0039989622,"Date":1764282593000,"Link":"https:\/\/www.bloomberg.com\/news\/articles\/2025-11-27\/asian-stocks-to-ebb-as-global-equity-rally-stalls-markets-wrap","Industry":"General Market"}],"ArticleCount":1,"ImpactfulNews":[{"Headline":"Asian Stocks Ebb as Global Rally Loses Momentum: Markets Wrap","Article":"Asian stocks wobbled at the open Friday as the sharp rebound in global equities over the past week showed signs of losing momentum.","SentimentScore":0.0039989622,"Date":null,"Link":null,"Industry":null}],"AvgSentimentScore":0.0039989622,"SentimentScore":0.001944202,"SentimentExplanation":"Explanation: The near-zero FinBERT score (~0.004) reflects a mixed-to-cautious outlook. The article

In [16]:
import pandas as pd
parsed = pd.DataFrame(json.loads(data))
print(f"\nFinal number of entries: {len(parsed)}")


Final number of entries: 3


In [17]:
parsed

,Industry,Date,News,ArticleCount,ImpactfulNews,AvgSentimentScore,SentimentScore,SentimentExplanation,index_ticker,feature_date,...,dollar_vol,log_dollar_vol,ret_vol_20d,ret_mom_20d,dollar_vol_rel_20d,ret_next,Date_mkt,MKT,SentimentScore_std,pred_ret_next
0,General Market,1764282593000,[{'Headline': 'Asian Stocks Ebb as Global Rall...,1,[{'Headline': 'Asian Stocks Ebb as Global Rall...,0.003999,0.001944,Explanation: The near-zero FinBERT score (~0.0...,SPY,1764115200000,...,4.885513e+10,24.612125,0.009678,-0.011216,0.819058,None,1764115200000,0.006903,0.001944,0.006819
1,Financials,1764286416000,[{'Headline': 'Oil in Worst Monthly Run Since ...,1,[{'Headline': 'Oil in Worst Monthly Run Since ...,-0.008944,0.002802,Near-neutral to slightly negative sentiment (-...,XLF,1764115200000,...,1.867467e+09,21.347849,0.008286,0.016510,0.740957,None,1764115200000,0.006903,0.002802,0.006444
2,Financials,1764287061000,[{'Headline': 'Gold Poised for Fourth Monthly ...,1,[{'Headline': 'Gold Poised for Fourth Monthly ...,-0.858359,-0.867723,The FinBERT score (~-0.86) signals strong nega...,XLF,1764115200000,...,1.867467e+09,21.347849,0.008286,0.016510,0.740957,None,1764115200000,0.006903,-0.867723,0.006516


In [18]:
parsed.iloc[0]

,0
Industry,General Market
Date,1764282593000
News,[{'Headline': 'Asian Stocks Ebb as Global Rall...
ArticleCount,1
ImpactfulNews,[{'Headline': 'Asian Stocks Ebb as Global Rall...
AvgSentimentScore,0.003999
SentimentScore,0.001944
SentimentExplanation,Explanation: The near-zero FinBERT score (~0.0...
index_ticker,SPY
feature_date,1764115200000
